In [1]:
import numpy as np
import keras
import pandas as pd
import h5py

Using TensorFlow backend.


In [2]:
# AS data file is in H5 document, using h5py package to read the file
f = h5py.File('SVHN_single_grey1.h5', 'r')
list(f.keys())

['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']

In [3]:
#Loading Train and Test data from the file
x_train = f.get('X_train')
y_train = f.get('y_train')
x_test = f.get('X_test')
y_test = f.get('y_test')

In [4]:
print('Number of samples in Training dataset: ', x_train.shape[0])
print('Number of samples in Test dataset: ', x_test.shape[0])
print('Dimensions of the image:', x_train[0].shape)

Number of samples in Training dataset:  42000
Number of samples in Test dataset:  18000
Dimensions of the image: (32, 32)


### KNN Classification

In [5]:
x_train_lst = []

for x in x_train:
    x = x.astype('float32')
    x /= 255
    x_train_lst.append(x.reshape(1024))

x_train_df = pd.DataFrame(x_train_lst)

In [6]:
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(x_train_df, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [8]:
#Preparing Test Data
x_test_lst = []

for x in x_test:
    x = x.astype('float32')
    x /= 255
    x_test_lst.append(x.reshape(1024))

x_test_df = pd.DataFrame(x_test_lst)

In [14]:
y_pred = classifier.predict(x_test_df.iloc[0:1000,])

In [15]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test[0:1000,], y_pred))
print(classification_report(y_test[0:1000,], y_pred))

[[77  8  1  2  2  2  8  1  7  9]
 [ 3 72  4  5  4  1  2  2  0  0]
 [10 18 41  5  1  1  2  4  4  6]
 [ 9 10 10 36  2  6  3  1  3  5]
 [11 14  3  2 58  1  2  2  2  1]
 [11 12  3 15  3 41  5  1  6  8]
 [13  7  2  2 13 10 38  0  7  2]
 [ 6 16  7  1  1  1  0 71  1  3]
 [22  9  4  3  3  6  8  2 39  4]
 [20  8  5  8  5  4  7  4 10 40]]
              precision    recall  f1-score   support

           0       0.42      0.66      0.52       117
           1       0.41      0.77      0.54        93
           2       0.51      0.45      0.48        92
           3       0.46      0.42      0.44        85
           4       0.63      0.60      0.62        96
           5       0.56      0.39      0.46       105
           6       0.51      0.40      0.45        94
           7       0.81      0.66      0.73       107
           8       0.49      0.39      0.44       100
           9       0.51      0.36      0.42       111

   micro avg       0.51      0.51      0.51      1000
   macro avg       

#### With KNN the prediction accuracy was between 50-60%

### Deep Neural Network Classifier

In [10]:
x_train_nn = x_train.value
x_test_nn = x_test.value

In [11]:
x_train_nn = x_train_nn.astype('float32')
x_test_nn = x_test_nn.astype('float32')

x_train_nn /= 255
x_test_nn /= 255

In [12]:
x_train_nn = x_train_nn.reshape(x_train_nn.shape[0], 32, 32, 1).astype('float32')
x_test_nn = x_test_nn.reshape(x_test_nn.shape[0], 32, 32, 1).astype('float32')

In [16]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape

#### Implementing a Deep Neural Network Classifier with
1. RELU Activations
2. Batch Normalisation
3. Cost Functions
4. Optimizers

In [18]:
# Define model
model = Sequential()

# 1st Conv Layer
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 1)))
model.add(Activation('relu'))

# 2nd Conv Layer
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

#Batch Normalisation
model.add(keras.layers.BatchNormalization())

# Prediction Layer
model.add(Dense(output_dim=10, init='he_normal', bias=True))
model.add(Activation('softmax'))

# Loss and Optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
# Store Training Results
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]

# Train the model
model.fit(x_train_nn, y_train.value, batch_size=32, epochs=10,
           validation_data=(x_test_nn, y_test.value), callbacks=callback_list)

C:\AIML\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, kernel_initializer="he_normal", use_bias=True)`
C:\Users\ssbru\AppData\Roaming\Python\Python36\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


Train on 42000 samples, validate on 18000 samples
Epoch 1/10
42000/42000 [==============================] - 212s 5ms/step - loss: 0.9241 - acc: 0.7117 - val_loss: 0.6330 - val_acc: 0.8059
Epoch 2/10
42000/42000 [==============================] - 207s 5ms/step - loss: 0.5324 - acc: 0.8368 - val_loss: 0.5124 - val_acc: 0.8456
Epoch 3/10
42000/42000 [==============================] - 181s 4ms/step - loss: 0.4432 - acc: 0.8648 - val_loss: 0.5657 - val_acc: 0.8260
Epoch 4/10
42000/42000 [==============================] - 181s 4ms/step - loss: 0.3757 - acc: 0.8822 - val_loss: 0.4549 - val_acc: 0.8697
Epoch 5/10
42000/42000 [==============================] - 181s 4ms/step - loss: 0.3306 - acc: 0.8955 - val_loss: 0.5295 - val_acc: 0.8505
Epoch 6/10
42000/42000 [==============================] - 187s 4ms/step - loss: 0.2942 - acc: 0.9080 - val_loss: 0.4885 - val_acc: 0.8644
Epoch 7/10
42000/42000 [==============================] - 184s 4ms/step - loss: 0.2579 - acc: 0.9182 - val_loss: 0.5372 - 

In [20]:
y_pred_nn = model.predict(x_test_nn)

In [40]:
y_pred_lst = np.argmax(y_pred_nn, axis=1)
y_pred_df = pd.DataFrame(y_pred_lst)
y_pred_df.head(5)

,0
0,1
1,7
2,2
3,9
4,0


In [42]:
print(confusion_matrix(y_test, y_pred_df))  
print(classification_report(y_test, y_pred_df))

[[1694   17    7    7   12    4   24    9   15   25]
 [  69 1579   11   28   64   12   10   17   23   15]
 [  26    7 1566   36   39    7    5   30   24   63]
 [  25   20   13 1402   19  125   20    9   56   30]
 [  19   40   13   24 1616   14   23    8   22   33]
 [  13   13    8   79   12 1517   63   11   28   24]
 [  60   16    7   31   24   61 1543    1   69   20]
 [  51   64   44   41   16   20   14 1535    9   14]
 [  35   25    9   58   23   41   67    5 1505   44]
 [  76   27   19   39   16   38   16   12   31 1530]]
              precision    recall  f1-score   support

           0       0.82      0.93      0.87      1814
           1       0.87      0.86      0.87      1828
           2       0.92      0.87      0.89      1803
           3       0.80      0.82      0.81      1719
           4       0.88      0.89      0.88      1812
           5       0.82      0.86      0.84      1768
           6       0.86      0.84      0.85      1832
           7       0.94      0.85   

### Summary

##### Accuracy of KNN ML: 50-60%
##### Accuracy of DL ML: 86%

##### Image classification is better done by Deep Learning model 

Accuracy of Deep Learning Model was approximately 30% higher than Traditional Machine Learning (KNN) model.
KNN model computation took longer than Deep Learning computations. 
As we run DL model in batches of data, in the first epoch itself we could see the accuracy and loss and hot the model performs.
As we can not do feature analysis on pixel values, we can not improve the efficiency and accuracy of the KNN model.
DL perfroms better in places where the EDL is not easily possible. 